<a href="https://colab.research.google.com/github/sreebalajisree/Fake_News_Detection/blob/main/Fake_News_Detection_Using_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [41]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [42]:
!pip install texthero

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.3.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.8 MB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Using cached srsly-1.0.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (208 kB)
  Using cached thinc-7.4.6-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3

In [43]:
!pip install Cython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-3.4.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.4 MB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached thinc-8.1.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (806 kB)
  Using cached srsly-2.4.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (490 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.2
    Uninstalling catalogue-1.0.2:
      Successfully uninstalled catalogue-1.0.2
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.6
    Uninstalling srsly-1.0.6:
      Successfully uninstalled srsly-1.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.6
    Uninstalling thinc-7.4.6:
      Successfully uninstalled thinc-7.4.6
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.8
    Uninstalling spacy-2.3.8

In [45]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [46]:
#Basic Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Sk-Learn libraries
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score,confusion_matrix,precision_score,recall_score, accuracy_score, classification_report

#Text-hero for Data Cleaning, processing
import texthero as hero
from texthero import visualization

#Sci-py library
from scipy.stats import pearsonr, spearmanr

#Transformer Libraries
from simpletransformers.t5 import T5Model
from transformers.data.metrics.squad_metrics import compute_exact, compute_f1

#Other libraries
import json
from datetime import datetime
from pprint import pprint
from statistics import mean

# Load the dataset

In [47]:
fake_train_covid = pd.read_excel("/content/sample_data/data/Constraint_English_Train.xlsx")
fake_valid_covid = pd.read_excel("/content/sample_data/data/Constraint_English_Val.xlsx")
fake_test_covid = pd.read_excel("/content/sample_data/data/Constraint_English_Test_without_labels.xlsx")
fake_external1 = pd.read_excel("/content/sample_data/data/external_1.xlsx")
fake_external2 = pd.read_excel("/content/sample_data/data/external_2.xlsx")
fake_train = pd.read_excel("/content/sample_data/data/new_train_data_all_topic.xlsx")
fake_valid = pd.read_excel("/content/sample_data/data/new_valid_data_all_topic.xlsx")
fake_test = pd.read_excel("/content/sample_data/data/new_test_data_all_topic.xlsx")

# Read the top 5 values from the datasets

In [48]:
fake_train_covid.head()

,id,tweet,label
0,1.0,The CDC currently reports 99031 deaths. In gen...,real
1,2.0,States reported 1121 deaths a small rise from ...,real
2,3.0,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4.0,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5.0,Populous states can generate large case counts...,real


In [49]:
fake_valid_covid.head()

,id,tweet,label
0,1.0,Chinese converting to Islam after realising th...,fake
1,2.0,11 out of 13 people (from the Diamond Princess...,fake
2,3.0,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4.0,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5.0,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [50]:
fake_test_covid.head()

,id,tweet
0,1.0,Our daily update is published. States reported...
1,2.0,Alfalfa is the only cure for COVID-19.
2,3.0,President Trump Asked What He Would Do If He W...
3,4.0,States reported 630 deaths. We are still seein...
4,5.0,This is the sixth time a global health emergen...


# Data preprocessing

In [51]:
def data_process(data):
  for i, label in enumerate(data['label']):
    if(data['label'][i] == "real"):
      data['label'][i] = '1'
    else:
      data['label'][i] = '0'

In [52]:
fake_train1 = pd.concat([fake_train_covid['tweet'], fake_train_covid['label']], axis=1)
fake_valid1 = pd.concat([fake_valid_covid['tweet'], fake_valid_covid['label']], axis=1)
fake_external1_df = pd.concat([fake_external1['tweet'], fake_external1['label']], axis=1)

In [53]:
# Call the data_processing method
data_process(fake_train1)
data_process(fake_valid1)

In [54]:
fake_train1 = pd.concat([fake_train1, fake_valid1], ignore_index=True)
fake_train1.head()

,tweet,label
0,The CDC currently reports 99031 deaths. In gen...,1
1,States reported 1121 deaths a small rise from ...,1
2,Politically Correct Woman (Almost) Uses Pandem...,0
3,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,Populous states can generate large case counts...,1


In [55]:
# Verify the dtype of the dataframe
print(fake_train1.dtypes)
print(fake_external1_df.dtypes)

tweet    object
label    object
dtype: object
tweet    object
label     int64
dtype: object


In [56]:
fake_train1.label.value_counts()

1    4480
0    4080
Name: label, dtype: int64

In [57]:
"""
Clean the train dataset using texthero library
"""
df_cleaned_tweet_fake_train = pd.DataFrame()
df_cleaned_tweet_fake_train['tweet'] = hero.clean(fake_train1['tweet'])
df_cleaned_tweet_fake_train = pd.concat([df_cleaned_tweet_fake_train['tweet'], fake_train1['label']], axis=1)

In [58]:
"""
Clean the valid dataset using texthero library
"""
df_cleaned_tweet_fake_valid = pd.DataFrame()
df_cleaned_tweet_fake_valid['tweet'] = hero.clean(fake_valid1['tweet'])
df_cleaned_tweet_fake_valid = pd.concat([df_cleaned_tweet_fake_valid['tweet'], fake_valid1['label']], axis=1)

In [59]:
"""
Clean the external dataset using texthero library
"""
df_cleaned_tweet_fake_external = pd.DataFrame()
df_cleaned_tweet_fake_external['tweet'] = hero.clean(fake_external1_df['tweet'])
df_cleaned_tweet_fake_external = pd.concat([df_cleaned_tweet_fake_external['tweet'], fake_external1['label']], axis=1)

In [60]:
binary_train_df = pd.DataFrame({
    'prefix': ["binary classification" for i in range(len(df_cleaned_tweet_fake_train))],
    'input_text': df_cleaned_tweet_fake_train.tweet.str.replace('\n', ' '),
    'target_text': df_cleaned_tweet_fake_train.label.astype(str),
})

print(binary_train_df.head())

                  prefix                                         input_text  \
0  binary classification  cdc currently reports deaths general discrepan...   
1  binary classification  states reported deaths small rise last tuesday...   
2  binary classification  politically correct woman almost uses pandemic...   
3  binary classification  indiafightscorona covid testing laboratories i...   
4  binary classification  populous states generate large case counts loo...   

  target_text  
0           1  
1           1  
2           0  
3           1  
4           1  


In [61]:
binary_valid_df = pd.DataFrame({
    'prefix': ["binary classification" for i in range(len(df_cleaned_tweet_fake_valid))],
    'input_text': df_cleaned_tweet_fake_valid.tweet.str.replace('\n', ' '),
    'target_text': df_cleaned_tweet_fake_valid.label.astype(str),
})

print(binary_valid_df.head())

                  prefix                                         input_text  \
0  binary classification  chinese converting islam realising muslim affe...   
1  binary classification  people diamond princess cruise ship intially t...   
2  binary classification       covid caused bacterium virus treated aspirin   
3  binary classification  mike pence rnc speech praises donald trump' co...   
4  binary classification  sky edconwaysky explains latest covid19 data g...   

  target_text  
0           0  
1           0  
2           0  
3           0  
4           1  


In [62]:
binary_external_df = pd.DataFrame({
    'prefix': ["binary classification" for i in range(len(df_cleaned_tweet_fake_external))],
    'input_text': df_cleaned_tweet_fake_external.tweet.str.replace('\n', ' '),
    'target_text': df_cleaned_tweet_fake_external.label.astype(str),
})

print(binary_external_df.head())

                  prefix                                         input_text  \
0  binary classification  travellers adhere strict hygiene measures wash...   
1  binary classification  first time post war history epidemics reversal...   
2  binary classification  understing japanese doctor offers excellent ad...   
3  binary classification  drinking lemon water could kill virus due vita...   
4  binary classification  coronavirus hoax fake virus pandemic fabricate...   

  target_text  
0           1  
1           1  
2           0  
3           0  
4           0  


# View the cleaned data top 5 values

In [63]:
df_cleaned_tweet_fake_train.head()

,tweet,label
0,cdc currently reports deaths general discrepan...,1
1,states reported deaths small rise last tuesday...,1
2,politically correct woman almost uses pandemic...,0
3,indiafightscorona covid testing laboratories i...,1
4,populous states generate large case counts loo...,1


In [64]:
df_cleaned_tweet_fake_valid.head()

,tweet,label
0,chinese converting islam realising muslim affe...,0
1,people diamond princess cruise ship intially t...,0
2,covid caused bacterium virus treated aspirin,0
3,mike pence rnc speech praises donald trump' co...,0
4,sky edconwaysky explains latest covid19 data g...,1


In [65]:
df_cleaned_tweet_fake_external.head()

,tweet,label
0,travellers adhere strict hygiene measures wash...,1
1,first time post war history epidemics reversal...,1
2,understing japanese doctor offers excellent ad...,0
3,drinking lemon water could kill virus due vita...,0
4,coronavirus hoax fake virus pandemic fabricate...,0


# Train the dataset with T5 base

In [66]:
model_args = {
    "max_seq_length": 196,
    "train_batch_size": 16,
    "eval_batch_size": 64,
    "num_train_epochs": 1,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 15000,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": False,
    "fp16": False,
    "save_steps": -1,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "wandb_project": "T5 tasks - Binary classification",
}

model = T5Model("t5", "t5-base", args=model_args, use_cuda=False)

model.train_model(binary_train_df, eval_data=binary_valid_df)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


  0%|          | 0/8560 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Training loss,▂▄▃▄▁█▄▂▁▂
eval_loss,▁
global_step,▁▂▂▃▄▅▅▆▇▇█
lr,▁▁▁▁▁▁▁▁▁▁
train_loss,▁
Training loss,0.11667
eval_loss,0.08342
global_step,535
lr,0.001
train_loss,0.15666


Running Epoch 0 of 1:   0%|          | 0/535 [00:00<?, ?it/s]

  0%|          | 0/2140 [00:00<?, ?it/s]

(535,
 {'global_step': [535],
  'eval_loss': [0.03957652114331722],
  'train_loss': [0.1043667420744896]})

# Model score functions

In [67]:
def f1(truths, preds):
    return mean([compute_f1(truth, pred) for truth, pred in zip(truths, preds)])


def exact(truths, preds):
    return mean([compute_exact(truth, pred) for truth, pred in zip(truths, preds)])


def pearson_corr(preds, labels):
    return pearsonr(preds, labels)[0]


def spearman_corr(preds, labels):
    return spearmanr(preds, labels)[0]


# Model Validation

In [68]:
# Model arguments for testing
model_args = {
    "overwrite_output_dir": True,
    "max_seq_length": 196,
    "eval_batch_size": 32,
    "use_multiprocessing": False,
    "num_beams": None,
    "do_sample": True,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 3,
}

In [69]:
# Load the trained model
model = T5Model(model_name="/content/outputs/best_model", model_type= 't5', args=model_args, use_cuda=False)

In [70]:
binary_external_df

,prefix,input_text,target_text
0,binary classification,travellers adhere strict hygiene measures wash...,1
1,binary classification,first time post war history epidemics reversal...,1
2,binary classification,understing japanese doctor offers excellent ad...,0
3,binary classification,drinking lemon water could kill virus due vita...,0
4,binary classification,coronavirus hoax fake virus pandemic fabricate...,0
...,...,...,...
1149,binary classification,ms n lives wuhan china takes special care well...,0
1150,binary classification,based know persons higher risk severe illness ...,1
1151,binary classification,allocation complex delicate choice choice prio...,1
1152,binary classification,"russian aid delivered italy ""russia helping it...",0


In [71]:
# Prepare the data for testing using valid_dataset
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(binary_valid_df["prefix"].tolist(), binary_valid_df["input_text"].tolist())
]
truth = binary_valid_df["target_text"].tolist()
tasks = binary_valid_df["prefix"].tolist()

In [72]:
# Get the model predictions
preds = model.predict(to_predict)

Generating outputs:   0%|          | 0/67 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3672: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/6420 [00:00<?, ?it/s]

In [73]:
# Saving the predictions if needed
with open(f"/content/sample_data/predictions/predictions_{datetime.now()}.txt", "w") as f:
    for i, text in enumerate(binary_valid_df["input_text"].tolist()):
        f.write(str(text) + "\n\n")

        f.write("Truth:\n")
        f.write(truth[i] + "\n\n")

        f.write("Prediction:\n")
        for pred in preds[i]:
            f.write(str(pred) + "\n")
        f.write(
            "________________________________________________________________________________\n"
        )

In [74]:
# Taking only the first prediction
preds = [pred[0] for pred in preds]
binary_valid_df["predicted"] = preds

# Evaluating the tasks separately
output_dict = {
    "binary classification": {
        "truth": [],
        "preds": [],
    },
  }

results_dict = {}

In [75]:
output_dict

{'binary classification': {'truth': [], 'preds': []}}

In [76]:
for task, truth_value, pred in zip(tasks, truth, preds):
    output_dict[task]["truth"].append(truth_value)
    output_dict[task]["preds"].append(pred)

print("-----------------------------------")
print("Results: ")
for task, outputs in output_dict.items():
        task_truth = [int(t) for t in output_dict[task]["truth"]]
        task_preds = [int(p) for p in output_dict[task]["preds"]]
        results_dict[task] = {
                "F1 Score": f1_score(task_truth, task_preds, average='weighted'),
                "Accuracy Score": accuracy_score(task_truth, task_preds),
          }
        print(f"Scores for {task}:")
        print(f"F1 score: {results_dict[task]['F1 Score']}")
        print(f"Accuracy Score: {results_dict[task]['Accuracy Score']}")

-----------------------------------
Results: 
Scores for binary classification:
F1 score: 0.9369271900343338
Accuracy Score: 0.9369158878504673


# Save the results

In [77]:
with open(f"/content/sample_data/results/result_{datetime.now()}.json", "w") as f:
    json.dump(results_dict, f)